In [ ]:
import h5py
for i in os.listdir("MICCAI2024/alls_data/unlab/"):
    path = "MICCAI2024/alls_data/unlab/" + i
    h5file = h5py.File(path, 'r')
    img = h5file['image']
    print(np.max(img), np.min(img))

In [ ]:
import numpy as np
from glob import glob
from tqdm import tqdm
import h5py
import nibabel as nib
import pandas as pd
import pdb
import SimpleITK as sitk
from skimage import transform, measure
import os
import pydicom
import matplotlib.pyplot as plt
%matplotlib inline
import pdb
import SimpleITK as sitk
from skimage import transform, measure
import os
import pydicom
import matplotlib.pyplot as plt
%matplotlib inline
output_size = [96, 96 ,96]
def ImageResample(sitk_image, new_spacing = [0.5,]*3, is_label = False):
    '''
    sitk_image:
    new_spacing: x,y,z
    is_label: if True, using Interpolator `sitk.sitkNearestNeighbor`
    '''
    size = np.array(sitk_image.GetSize())
    spacing = np.array(sitk_image.GetSpacing())
    new_spacing = np.array(new_spacing)
    new_size = size * spacing / new_spacing
    new_spacing_refine = size * spacing / new_size
    new_spacing_refine = [float(s) for s in new_spacing_refine]
    new_size = [int(s) for s in new_size]
    if not is_label:
        print(size)
        print(new_size)
        print(spacing)
        print(new_spacing_refine)
        # return  sitk_image
    resample = sitk.ResampleImageFilter()
    resample.SetOutputDirection(sitk_image.GetDirection())
    resample.SetOutputOrigin(sitk_image.GetOrigin())
    resample.SetSize(new_size)
    resample.SetOutputSpacing(new_spacing_refine)
    # print(2)
    # print(sitk_image.GetDirection())
    # print(sitk_image.GetOrigin())
    
    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkBSpline)
    newimage = resample.Execute(sitk_image)
    return newimage


def set_window_wl_ww(tensor):
    sl_window = [500, 2000]
    [wl,ww] = sl_window
    tensor[tensor < wl] = wl
    tensor[tensor > ww] = ww
    tensor = (tensor - wl) / (ww - wl)
    ### min max Normalization
    return tensor

def crop_roi(image, label):
    assert(image.shape == label.shape)
    print (image.shape)
    ### crop based on lung segmentation
    w, h, d = label.shape

    tempL = np.nonzero(label)
    minx, maxx = np.min(tempL[0]), np.max(tempL[0])
    miny, maxy = np.min(tempL[1]), np.max(tempL[1])

    px = max(output_size[0] - (maxx - minx), 0) // 2
    py = max(output_size[1] - (maxy - miny), 0) // 2
    minx = max(minx - px - 25, 0) #np.random.randint(5, 10)
    maxx = min(maxx + px + 25, w) #np.random.randint(5, 10)
    miny = max(miny - py - 25, 0)
    maxy = min(maxy + py + 25, h)
    
    image = image[minx:maxx, miny:maxy,:].astype(np.float32)
    label = label[minx:maxx, miny:maxy,:].astype(np.float32)
    return image, label

root = "./origin_data/"
img_data = glob(root + 'lab/Images/*.nii.gz')
# print(img_data[0])
unlab_data = glob(root + 'unlab/*.nii.gz')
val_data = glob(root + 'val/*.nii.gz')
print(img_data)

In [ ]:
def cal_Norm(paths):
    voxels_all = []
    for item in tqdm(img_data):
        flag = 0
        name_image = str(item)
        name_label = name_image.replace('Images', 'Masks').replace(".nii.gz", '_Mask.nii.gz')

        itk_img = sitk.ReadImage(name_image)
        file_size_bytes = os.path.getsize(name_image)

        file_size_mb = file_size_bytes / (1024 * 1024)
        if file_size_mb < 70:
            pass
        else:
            itk_img = ImageResample(itk_img, [0.6,]*3)
            sitk.WriteImage(itk_img, './temp_img.nii.gz')
            flag = 1
        image = sitk.GetArrayFromImage(itk_img)
        image = np.transpose(image, (2,1,0))
        
        itk_label = sitk.ReadImage(name_label)
        if flag == 1:
            itk_label = ImageResample(itk_label, [0.6,]*3 , is_label = True)
            sitk.WriteImage(itk_label, './temp_label.nii.gz')
        label = sitk.GetArrayFromImage(itk_label)
        label = np.transpose(label, (2,1,0))
        
        mask = label > 0
        # print(image.shape, label.shape)
        voxels = image[mask]  
        
        voxels_all.extend(voxels)
    
    mean = np.mean(voxels_all)
    std = np.std(voxels_all)
    percentile_99_5 = np.percentile(voxels_all, 99.5)
    percentile_00_5 = np.percentile(voxels_all, 0.5)
    return percentile_00_5, percentile_99_5, mean, std

# print(cal_Norm(img_data))
        
percentile_00_5, percentile_99_5 = [167.0, 3095.0]

In [ ]:
# map
size = [i for i in range(33)]
zidian = {}
ni_zidian = {}
zidian[1] = 0
zidian[0] = 0
index = 1
for a in range(11, 19):
    zidian[a] = index
    ni_zidian[index] = a
    index += 1
for b in range(21, 29):
    zidian[b] = index
    ni_zidian[index] = b  
    index += 1
for c in range(31, 39):
    zidian[c] = index
    ni_zidian[index] = c
    index += 1
for d in range(41, 49):
    zidian[d] = index
    ni_zidian[index] = d
    index += 1
print(zidian)
print(ni_zidian)



In [3]:
top = 2500
bottom = 500 

In [ ]:
# img 
# "distill_match/data/MICCAI2024/origin_data/lab/Images/STS24_Train_Labeled_0001.nii.gz"
root = './origin_data/lab/Images/STS24_Train_Labeled_0001.nii.gz'
print(root[-11:-7])

In [5]:
for item in img_data:
    # First, Resample
    # 读取为numpy, 
    name_image = str(item)
    name_label = name_image.replace('Images', 'Masks').replace(".nii.gz", '_Mask.nii.gz')

    itk_img = sitk.ReadImage(name_image)
    image = sitk.GetArrayFromImage(itk_img)
    image = np.transpose(image, (2,1,0))

    
    itk_label = sitk.ReadImage(name_label)
    label = sitk.GetArrayFromImage(itk_label)
    label = np.transpose(label, (2,1,0))

    image[image > top] = top
    image[image < bottom] = bottom
    # 0 - 1
    image =  (image - bottom)/(top - bottom) 
    
    if "_0001" in name_image:
        label[label == 21] = 0
        label[label == 23] = 14
    if "_0028" in name_image:
        label[label == 14] == 50
        label[label == 15] == 14
        label[label == 50] == 15
    for cls in np.unique(label):
        label[label == cls] = zidian[cls]
        
    f = h5py.File(('./alls_data_no/lab/imgs_' + name_image[-11:-7] + '_norm.h5'), 'w')
    f.create_dataset('image', data=image, compression="gzip")
    f.create_dataset('label', data=label, compression="gzip")
    f.close()

In [ ]:
for item in tqdm(unlab_data):
    name_image = str(item)
    itk_img = sitk.ReadImage(name_image)
    image = sitk.GetArrayFromImage(itk_img)
    image = np.transpose(image, (2,1,0))
    
    image[image > top] = top
    image[image < bottom] = bottom
    # 0 - 1
    image =  (image - bottom)/(top - bottom) 

    f = h5py.File(('./alls_data_no/unlab/imgs_' + name_image[-11:-7] + '_norm.h5'), 'w')
    f.create_dataset('image', data=image, compression="gzip")
    f.close()

In [ ]:
# for val
for item in tqdm(val_data):
    name_image = str(item)
    itk_img = sitk.ReadImage(name_image)
    shape = np.array(sitk.GetArrayFromImage(itk_img).shape)
    image = sitk.GetArrayFromImage(itk_img)
    image = np.transpose(image, (2,1,0))
    
    image[image > top] = top
    image[image < bottom] = bottom
    # 0 - 1
    image =  (image - bottom)/(top - bottom) 
    
    f = h5py.File(('./alls_data_no/val/imgs_' + name_image[-11:-7] + '_norm.h5'), 'w')
    f.create_dataset('image', data=image, compression="gzip")
    f.create_dataset('shape', data=shape)

    f.close()

In [ ]:
img_data = ['MICCAI2024/origin_data/lab/Images/STS24_Train_Labeled_0001.nii.gz']
for item in img_data:
    # First, Resample
    # 读取为numpy, 
    flag = 0
    name_image = str(item)
    name_label = name_image.replace('Images', 'Masks').replace(".nii.gz", '_Mask.nii.gz')

    itk_img = sitk.ReadImage(name_image)
    image = sitk.GetArrayFromImage(itk_img)
    image = np.transpose(image, (2,1,0))

    
    itk_label = sitk.ReadImage(name_label)

    itk_label = ImageResample(itk_label, [0.6, 0.6, 0.45] ,is_label = True)
    label = sitk.GetArrayFromImage(itk_label)
    label = np.transpose(label, (2,1,0))

    percentile_99_5 = np.percentile(image, 99.5)
    percentile_00_5 = np.percentile(image, 0.5)
    image = np.clip(image, percentile_00_5, percentile_99_5)
    
    image = (image - np.mean(image)) / np.std(image)
    
    print(np.max(image), np.min(image))
    
    label[label == 21] = 0
    label[label == 23] = 14
    for cls in np.unique(label):
        label[label == cls] = zidian[cls]


    f = h5py.File(('MICCAI2024/alls_data/lab/imgs' + name_image[97:102] + '_norm.h5'), 'w')
    f.create_dataset('image', data=image, compression="gzip")
    f.create_dataset('label', data=label, compression="gzip")
    f.close()

In [ ]:
for item in val_data:
    name_image = str(item)
    itk_img = sitk.ReadImage(name_image)
    image = sitk.GetArrayFromImage(itk_img)
    print(name_image)
    print(image.shape)